In [1]:
pip install spaCy

Note: you may need to restart the kernel to use updated packages.


In [2]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     -- ------------------------------------- 0.8/12.8 MB 8.5 MB/s eta 0:00:02
     ------------------ --------------------- 6.0/12.8 MB 19.5 MB/s eta 0:00:01
     --------------------------------------  12.6/12.8 MB 25.5 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 22.3 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
# Standard libraries
import pandas as pd
import re

# spaCy
import spacy

# Loading the English pipeline
nlp = spacy.load("en_core_web_sm")


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
df = pd.read_csv("data/dataset_resume/Resume/Resume.csv") 
df.head(15)

,ID,Resume_str,Resume_html,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR
5,11592605,HR GENERALIST Summary Dedic...,"<div class=""fontsize fontface vmargins hmargin...",HR
6,25824789,HR MANAGER Summary HUMAN RES...,"<div class=""fontsize fontface vmargins hmargin...",HR
7,15375009,HR MANAGER Professional Summa...,"<div class=""fontsize fontface vmargins hmargin...",HR
8,11847784,HR SPECIALIST Summary Posses...,"<div class=""fontsize fontface vmargins hmargin...",HR
9,32896934,HR CLERK Summary Translates ...,"<div class=""fontsize fontface vmargins hmargin...",HR


In [6]:
df["Resume_html"]

0       <div class="fontsize fontface vmargins hmargin...
1       <div class="fontsize fontface vmargins hmargin...
2       <div class="fontsize fontface vmargins hmargin...
3       <div class="fontsize fontface vmargins hmargin...
4       <div class="fontsize fontface vmargins hmargin...
                              ...                        
2479    <div class="fontsize fontface vmargins hmargin...
2480    <div class="fontsize fontface vmargins hmargin...
2481    <div class="fontsize fontface vmargins hmargin...
2482    <div class="fontsize fontface vmargins hmargin...
2483    <div class="fontsize fontface vmargins hmargin...
Name: Resume_html, Length: 2484, dtype: object

In [16]:
import re

def clean_html(text):
    text = re.sub(r"<[^>]+>", " ", str(text))
    return re.sub(r"\s+", " ", text).strip()

In [17]:
df["Resume_cleaned"] = df["Resume_html"].apply(strip_html)

In [18]:
df

,ID,Resume_str,Resume_html,Category,Resume_cleaned,Experience_sentences
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,[HR ADMINISTRATOR/MARKETING ASSOCIATE\n\nHR AD...
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR,"HR SPECIALIST, US HR OPERATIONS ...","[HR SPECIALIST, US HR OPERATIONS Summary..."
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR,HR DIRECTOR Summary Over 2...,[HR DIRECTOR Summary Over 20 years ...
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR,HR SPECIALIST Summary Dedica...,"[HR SPECIALIST Summary Dedicated, Dri..."
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR,HR MANAGER Skill Highlights ...,[HR MANAGER Skill Highlights ...
...,...,...,...,...,...,...
2479,99416532,RANK: SGT/E-5 NON- COMMISSIONED OFFIC...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION,RANK: SGT/E-5 NON- COMMISSIONED OFFIC...,"[Summary Motivated logistics specialist, ..."
2480,24589765,"GOVERNMENT RELATIONS, COMMUNICATIONS ...","<div class=""fontsize fontface vmargins hmargin...",AVIATION,"GOVERNMENT RELATIONS, COMMUNICATIONS ...","[GOVERNMENT RELATIONS, COMMUNICATIONS AND ORGA..."
2481,31605080,GEEK SQUAD AGENT Professional...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION,GEEK SQUAD AGENT Professional...,[GEEK SQUAD AGENT Professional Profile...
2482,21190805,PROGRAM DIRECTOR / OFFICE MANAGER ...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION,PROGRAM DIRECTOR / OFFICE MANAGER ...,[PROGRAM DIRECTOR / OFFICE MANAGER S...


In [19]:
keywords = [
    "experience", "project", "management", "responsibility",
    "work history", "work experience", "job description",
    "role", "tasks", "positions", "certification", "abilities",
    "technical skills", "summary", "profile", "accomplishments"
]

def extract_key_sentences(text, keywords=keywords, nlp=nlp):
    """Extract sentences that contain any of the keywords from the text."""
    if pd.isna(text): return []
    text = clean_html(text)
    doc = nlp(text)
    return [sent.text.strip() for sent in doc.sents if any(k in sent.text.lower() for k in keywords)]



In [21]:
# Apply extraction to your DataFrame
df["Experience_sentences"] = df["Resume_html"].apply(extract_key_sentences)

In [13]:
df

,ID,Resume_str,Resume_html,Category,Resume_cleaned,Experience_sentences
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,[HR ADMINISTRATOR/MARKETING ASSOCIATE\n\nHR AD...
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR,"HR SPECIALIST, US HR OPERATIONS ...","[HR SPECIALIST, US HR OPERATIONS Summary..."
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR,HR DIRECTOR Summary Over 2...,[HR DIRECTOR Summary Over 20 years ...
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR,HR SPECIALIST Summary Dedica...,"[HR SPECIALIST Summary Dedicated, Dri..."
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR,HR MANAGER Skill Highlights ...,[HR MANAGER Skill Highlights ...
...,...,...,...,...,...,...
2479,99416532,RANK: SGT/E-5 NON- COMMISSIONED OFFIC...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION,RANK: SGT/E-5 NON- COMMISSIONED OFFIC...,"[Summary Motivated logistics specialist, ..."
2480,24589765,"GOVERNMENT RELATIONS, COMMUNICATIONS ...","<div class=""fontsize fontface vmargins hmargin...",AVIATION,"GOVERNMENT RELATIONS, COMMUNICATIONS ...","[GOVERNMENT RELATIONS, COMMUNICATIONS AND ORGA..."
2481,31605080,GEEK SQUAD AGENT Professional...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION,GEEK SQUAD AGENT Professional...,[GEEK SQUAD AGENT Professional Profile...
2482,21190805,PROGRAM DIRECTOR / OFFICE MANAGER ...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION,PROGRAM DIRECTOR / OFFICE MANAGER ...,[PROGRAM DIRECTOR / OFFICE MANAGER S...


In [14]:
df["Experience_sentences"]

0       [HR ADMINISTRATOR/MARKETING ASSOCIATE\n\nHR AD...
1       [HR SPECIALIST, US HR OPERATIONS       Summary...
2       [HR DIRECTOR       Summary      Over 20 years ...
3       [HR SPECIALIST       Summary    Dedicated, Dri...
4       [HR MANAGER         Skill Highlights          ...
                              ...                        
2479    [Summary      Motivated logistics specialist, ...
2480    [GOVERNMENT RELATIONS, COMMUNICATIONS AND ORGA...
2481    [GEEK SQUAD AGENT         Professional Profile...
2482    [PROGRAM DIRECTOR / OFFICE MANAGER           S...
2483    [STOREKEEPER II       Professional Summary    ...
Name: Experience_sentences, Length: 2484, dtype: object